## 🔗 Open This Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DavidLangworthy/ds4s/blob/master/Day%205_%20Capstone%20–%20CO₂%20Emissions%20%26%20Global%20Temperature.ipynb)

# 🔥 Day 5 – Capstone Project: CO₂ and Climate  
### Telling the Big Story with Data

You’ve made it to the final day! Today is your chance to bring everything together — loading real data, analyzing trends, building a polished visualization, and telling a story that matters.

Your task: **show how human-caused CO₂ emissions relate to global temperature rise**.  
It’s a chance to take what you’ve learned and apply it to a real-world issue, from start to finish.

---

## 🧭 What You’ll Do

This is a **capstone project** — more open-ended than the earlier days. You’ll:
- Load and combine multiple datasets
- Design a compelling visualization (or two)
- Write a clear narrative explaining what you found
- Build a mini-report you could share with others

You’ll work more independently, but still have prompts, examples, and a solution notebook to reference.

---

## 🔧 Tools & Setup

You can use:
- `pandas` for data handling
- `matplotlib` or `plotly` for plotting (your choice!)
- Google Colab as always — nothing new to install beyond what you’ve used

---

## 📊 Datasets

You’ll work with:
- **Global CO₂ Emissions** (1900–2022): Emissions in gigatonnes per year from fossil fuels. Source: Our World in Data / Global Carbon Project.
- **Global Temperature Anomalies** (1880–2024): Annual average temperature change (°C) from NASA GISTEMP.

> We’ve pre-cleaned these so they’re easy to load and join on the `Year` column.

Optional: If you finish early, you can add a third dataset (like methane, sea level, or atmospheric CO₂ concentration).

---

## 🛠️ Lab: CO₂ and Climate – Capstone Story

Open the notebook titled **“CO₂ and Climate: Capstone Story.”** Here’s the flow:

---

### 1. Question & Planning

You’ll start with this guiding question:

> **How do global CO₂ emissions relate to global temperature change?**

You decide how to visualize it:
- A **dual-axis line plot**?
- Two charts side-by-side?
- Something else?

The notebook gives you some suggestions and example code blocks to build from.

---

### 2. Loading the Data

You’ll read in the two CSVs:
```python
df_co2 = pd.read_csv('global_co2.csv')
df_temp = pd.read_csv('global_temp.csv')
```

Check the columns, inspect the structure, and filter the years to get a shared range (e.g. 1880–2020).

---

### 3. Merging the Data

Join the datasets on the `Year` column:
```python
df = pd.merge(df_co2, df_temp, on='Year')
```

You should now have a DataFrame with:
- `Year`
- `CO2` (e.g. in gigatonnes)
- `Temperature` (°C anomaly)

---

### 4. Visualization – Dual Line Chart

You’ll create a plot with:
- CO₂ emissions on one y-axis
- Temperature on another y-axis
- Both plotted over time

Example using `matplotlib`:

```python
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(df['Year'], df['CO2'], color='brown', label='CO₂ Emissions')
ax2.plot(df['Year'], df['Temperature'], color='red', label='Temperature Anomaly')

ax1.set_ylabel('CO₂ Emissions (Gt/year)')
ax2.set_ylabel('Temperature Anomaly (°C)')
ax1.set_xlabel('Year')
ax1.set_title('Global CO₂ Emissions and Temperature (1900–2020)')
```

> 💡 You’ll add labels, a legend, maybe an annotation at 2020, and polish your chart. The goal is to clearly show how these two lines **rise together** — a powerful visual of cause and effect.

---

### 5. Optional Add-On

If you want more:
- Try a **scatter plot** of CO₂ vs. Temperature (each point = a year)
- Animate the scatter over time
- Try using Plotly for interactivity

But the dual-line chart is more than enough!

---

### 6. Telling the Story

You’ll write up your project like a short data report.

The notebook includes Markdown cells for:
- **Introduction** – What’s the question? Why does it matter?
- **Plot + Analysis** – What does your visualization show?
- **Conclusion** – What did you learn? What’s the message?

> Example:  
> “The data shows a clear link: as CO₂ emissions grew from 5 to 35 gigatonnes per year between 1950 and 2020, global temperatures rose ~1°C. The correlation is striking — and lines up with what climate science tells us.”

Be sure to cite your sources:
- CO₂ data: Our World in Data / Global Carbon Project  
- Temperature: NASA GISTEMP

---

## 🧪 Support & Solutions

- The **starter notebook** includes code stubs, suggestions, and space for your writing
- The **solution notebook** provides a full example with a polished chart and narrative

TAs and instructors are available for help if you get stuck — but this is your chance to build something on your own!

---

## ✅ What You’ll Walk Away With

By completing this project, you’ll demonstrate:
- Independent data analysis and cleaning
- Merging and visualizing multiple datasets
- Writing an engaging and informative narrative
- The ability to **communicate sustainability insights with code and clarity**

This is your final notebook — your **portfolio piece**. It tells a story that the world needs to hear.

Let’s go tell it.
```

---

Let me know if you’d like the final course summary, export-friendly versions, or the full Markdown collection bundled together!

In [ ]:
# Install libraries if needed
from pathlib import Path
import pandas as pd

# --- Starter Notebook elide: import the core data wrangling libraries ---

# Load global CO₂ emissions that ship with the repo
data_dir = Path.cwd() / "data"
co2_path = data_dir / "global_co2.csv"
df_co2 = pd.read_csv(co2_path).set_index("Year")

# --- Starter Notebook elide: read the CO₂ dataset and prepare the index ---


In [ ]:
# Load NASA temperature anomalies from the local copy
temperature_path = data_dir / "GLB.Ts+dSST.csv"
df_temp = (
    pd.read_csv(
        temperature_path,
        skiprows=1,
        na_values=["***"],
        usecols=["Year", "J-D"],
    )
    .rename(columns={"J-D": "TempAnomaly"})
    .dropna()
)
df_temp["TempAnomaly"] = pd.to_numeric(df_temp["TempAnomaly"])
df_temp = df_temp.set_index("Year")

# --- Starter Notebook elide: clean the temperature dataset and align the index ---


In [ ]:
# Notebook: Day 5 – Capstone: CO₂ Emissions & Temperature
import matplotlib.pyplot as plt

# Merge data
df_merged = df_co2.join(df_temp, how="inner")

# Dual-axis line plot
fig, ax1 = plt.subplots(figsize=(10, 5))

ax1.plot(df_merged.index, df_merged["CO2"], color="gray", label="CO₂ Emissions (Gt)")
ax1.set_ylabel("CO₂ Emissions (Gt)", color="gray")
ax1.tick_params(axis="y", labelcolor="gray")

ax2 = ax1.twinx()
ax2.plot(df_merged.index, df_merged["TempAnomaly"], color="red", label="Temperature Anomaly (°C)")
ax2.set_ylabel("Temperature Anomaly (°C)", color="red")
ax2.tick_params(axis="y", labelcolor="red")

plt.title("Global CO₂ Emissions and Temperature Anomaly (1900–2020)")
fig.tight_layout()
plt.show()

# --- Starter Notebook elide: configure the dual-axis plot and styling ---


In [ ]:
# Save the dual-axis plot for the course roadmap

from pathlib import Path


def find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "plots").exists() or (candidate / ".git").exists() or (candidate / "README.md").exists():
            return candidate
    return start


repo_root = find_repo_root(Path.cwd())
plots_dir = repo_root / "plots"
plots_dir.mkdir(parents=True, exist_ok=True)

plot_path = plots_dir / "day05_co2_vs_temperature.png"
fig.savefig(plot_path, dpi=300, bbox_inches="tight")
print(f"Saved plot to {plot_path.relative_to(repo_root)}")
